In [156]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [157]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'rating']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [158]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [159]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [160]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()
ratings.head()
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [161]:
def title_to_id(name):
    t = movies['title'] == name
    tt = movies[t]
    r = tt['movie_id'].values[0].item()
    return r

def id_to_title(_id):
    t = movies['movie_id'] == _id
    tt = movies[t]
    r = tt['title'].item()
    return r

In [162]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [163]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

#new_ratings = pd.merge(ratings, movies, on='movie_id')
#new_ratings.head(10)

In [164]:
#가장 인기있는 영화 30개(인기순)
movie_count = new_ratings.groupby('title')['title'].count()
movie_count.sort_values(ascending=False).head(30)


title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [165]:
new_ratings.head()


Silence of the Lambs, The (1991)                         2498
Jurassic Park (1993)                                     2413
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Godfather, The (1972)                                    2167

SyntaxError: invalid syntax (<ipython-input-165-97d9a62c8115>, line 4)

#3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Jurassic Park (1993)' , 
               'Silence of the Lambs, The (1991)', 
               'Men in Black (1997)' ,
               'Godfather, The (1972)' ,
               'Braveheart (1995)'
              ]

#my_favorite_id = []

#for i in my_favorite:
    #my_favorite_id.append(title_to_id(i))
    
my_favorite_id = [2413 , 2498, 2297 ,2167 ,2314]

# 9999999('jungho')이라는 user_id가 위 영화를 4회씩 봤다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['jungho']*5, 'movie_id': my_favorite_id, 'title': my_favorite, 'counts':[4]*5})

print(my_playlist)

if not ratings.isin({'user_id':['jungho']})['user_id'].any():  # user_id에 'jungho'이라는 데이터가 없다면
    new_ratings = new_ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

new_ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

In [166]:
my_favorite = ['avengers, the (1998)' , 'lord of the rings, the (1978)' ,'saving private ryan (1998)' ,'shawshank redemption, the (1994)' ,'jurassic park (1993)']

my_favorite_id = []

for i in my_favorite:
    my_favorite_id.append(title_to_id(i))
    
print(my_favorite_id)

my_movielist = pd.DataFrame({'user_id': ['jungho']*5, 'movie_id': my_favorite_id, 'counts':[5]*5})

if not ratings.isin({'user_id':['jungho']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

[2153, 2116, 2028, 318, 480]


,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,jungho,2153,5
1,jungho,2116,5
2,jungho,2028,5
3,jungho,318,5
4,jungho,480,5


In [183]:
# 고유한 유저, movie를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

len(user_unique)

#print(movie_to_idx)

6040

In [184]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jungho'])    
#print(artist_to_idx['black eyed peas'])

6039


In [185]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_id_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_ratings = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_ratings
else:
    print('movie_id column indexing Fail!!')

ratings.tail(15)

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts
1000198,6038,691,3
1000199,6038,2446,5
1000200,6038,48,5
1000201,6038,693,4
1000202,6038,289,4
1000203,6038,1030,3
1000205,6038,986,5
1000206,6038,311,5
1000207,6038,142,4
1000208,6038,26,4


In [177]:
#new_ratings=new_ratings.sort_values(by=['user_id'], axis=0)
#new_ratings.tail(10)

,user_id,movie_id,counts,title,genre
633800,6037,791,4,fatal attraction (1987),Thriller
718135,6037,1130,5,"nightmare on elm street, a (1984)",Horror
743849,6037,1100,3,fright night (1985),Comedy|Horror
836273,6037,3588,3,city of the living dead (paura nella citt� dei...,Horror
828959,6038,2604,3,airport 1975 (1974),Drama
0,6039,1007,4,Jurassic Park (1993),NaN
1,6039,2665,4,"Silence of the Lambs, The (1991)",NaN
2,6039,941,4,Men in Black (1997),NaN
3,6039,566,4,"Godfather, The (1972)",NaN
4,6039,1949,4,Braveheart (1995),NaN


In [178]:
#new_ratings[new_ratings["user_id"]==6039].index=["836477","836478","836479","836480","836481"]
#new_ratings.tail(10)

,user_id,movie_id,counts,title,genre
633800,6037,791,4,fatal attraction (1987),Thriller
718135,6037,1130,5,"nightmare on elm street, a (1984)",Horror
743849,6037,1100,3,fright night (1985),Comedy|Horror
836273,6037,3588,3,city of the living dead (paura nella citt� dei...,Horror
828959,6038,2604,3,airport 1975 (1974),Drama
0,6039,1007,4,Jurassic Park (1993),NaN
1,6039,2665,4,"Silence of the Lambs, The (1991)",NaN
2,6039,941,4,Men in Black (1997),NaN
3,6039,566,4,"Godfather, The (1972)",NaN
4,6039,1949,4,Braveheart (1995),NaN


In [179]:
#print(temp_user_ratings.shape)
#print(new_ratings.shape)
#print(temp_movie_ratings.shape)
#print(new_ratings.shape)

(836483,)
(836483, 5)
(836483,)
(836483, 5)


In [190]:
#4) CSR matrix를 직접 만들어 봅시다.
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

#print(num_user)
#print(num_movie)

#ratings
#ratings.rating
#print(type(ratings))

#print(ratings.count.shape)
#print(ratings.user_id.shape)
#print(ratings.movie_id.shape)
#print(csr_ratings.shape)

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
#shape= (num_user, num_movie))


In [191]:
#5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
from implicit.als import AlternatingLeastSquares
import os
import numpy as np


# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose
als_model.fit(csr_data_transpose)

In [193]:
#6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
ans = []
for v in my_favorite_id:
    jungho, m = user_to_idx['jungho'], movie_to_idx[v]
    jh_vector, m_vector = als_model.user_factors[jungho], als_model.item_factors[m]
    ans.append(np.dot(jh_vector, m_vector))




In [195]:
jungho, m = user_to_idx['jungho'], movie_to_idx[title_to_id('toy story (1995)')]
jh_vector, m_vector = als_model.user_factors[jungho], als_model.item_factors[m]
np.dot(jh_vector, m_vector)

0.05665382

In [197]:
#7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

def get_similar_movie(movie_name: str):
    m_id = title_to_id(movie_name)
    movie_id = movie_to_idx[m_id]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

a = get_similar_movie('avengers, the (1998)')
for i in a:
    print(id_to_title(i))

avengers, the (1998)
batman & robin (1997)
cutthroat island (1995)
chain reaction (1996)
dear god (1996)
bird on a wire (1990)
phantom, the (1996)
tarzan and the lost city (1998)
island of dr. moreau, the (1996)
shadow, the (1994)


In [200]:
#8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

user = user_to_idx['jungho']

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

a = [idx_to_movie[i[0]] for i in movie_recommended]
for i in a:
    print(id_to_title(i))

braveheart (1995)
schindler's list (1993)
silence of the lambs, the (1991)
good will hunting (1997)
men in black (1997)
american beauty (1999)
fugitive, the (1993)
terminator 2: judgment day (1991)
matrix, the (1999)
forrest gump (1994)
fargo (1996)
pulp fiction (1994)
dead man walking (1995)
back to the future (1985)
sling blade (1996)
patriot, the (2000)
goodfellas (1990)
hunt for red october, the (1990)
one flew over the cuckoo's nest (1975)
star wars: episode iv - a new hope (1977)


In [202]:

a = movie_to_idx[title_to_id('black hole, the (1979)')]
explain = als_model.explain(user, csr_data, itemid=a)

a = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]
a = [idx_to_movie[i[0]] for i in a]
for i in a:
    print(id_to_title(i))

if.... (1968)
raise the titanic (1980)
rosencrantz and guildenstern are dead (1990)
jean de florette (1986)
grand day out, a (1992)
